In [1]:
import sys
from datetime import datetime
import pandas as pd
import json


print("="*80)
print("DATA QUALITY COM SODA CORE")
print("="*80)
print(f"\nIniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Ambiente: {sys.platform}")
print(f"Python: {sys.version.split()}")


DATA QUALITY COM SODA CORE

Iniciado em: 2026-01-10 18:53:21
Ambiente: win32
Python: ['3.14.0', '(tags/v3.14.0:ebf955d,', 'Oct', '7', '2025,', '10:15:03)', '[MSC', 'v.1944', '64', 'bit', '(AMD64)]']


In [2]:
# ===== CARREGAR DADOS RAW =====
print("\n" + "="*80)
print("CARREGANDO DADOS RAW")
print("="*80)

customers_raw = '../data/raw/customers_raw.csv'
products_raw = '../data/raw/products_raw.json'
sales_raw = '../data/raw/sales_raw.csv'

try:
    customers_raw = pd.read_csv(customers_raw)
    print(f"customers_raw: {len(customers_raw):,} registros carregados")
except FileNotFoundError:
    print(f"Arquivo não encontrado: {customers_raw}")
    customers_raw = None

try:
    with open(products_raw, 'r') as f:
        products_raw = pd.DataFrame(json.load(f))
    print(f"products_raw: {len(products_raw):,} registros carregados")
except FileNotFoundError:
    print(f"Arquivo não encontrado: {products_raw}")
    products_raw = None

try:
    sales_raw = pd.read_csv(sales_raw)
    print(f"sales_raw: {len(sales_raw):,} registros carregados")
except FileNotFoundError:
    print(f"Arquivo não encontrado: {sales_raw}")
    sales_raw = None



CARREGANDO DADOS RAW
customers_raw: 5,000 registros carregados
products_raw: 10,000 registros carregados
sales_raw: 120,000 registros carregados


In [3]:
# ===== ANÁLISE DE COMPLETUDE =====
print("\n" + "="*80)
print("ANÁLISE DE COMPLETUDE (Missing Values)")
print("="*80)

def analyze_completeness(df, table_name):
    """Analisa valores faltantes em cada coluna"""
    if df is None:
        print(f"{table_name}: Tabela não carregada")
        return None
    
    print(f"\n{table_name}:")
    print(f"Total de linhas: {len(df):,}")
    
    missing_data = df.isnull().sum()
    completeness = (1 - missing_data / len(df)) * 100
    
    report = pd.DataFrame({
        'Campo': missing_data.index,
        'Nulos': missing_data.values,
        'Completude %': completeness.values.round(2)
    })
    
    # Filtrar apenas campos com problemas
    problematic = report[report['Nulos'] > 0].sort_values('Nulos', ascending=False)
    
    if len(problematic) > 0:
        print(f"\n CAMPOS COM VALORES FALTANTES:")
        for idx, row in problematic.iterrows():
            print(f"• {row['Campo']}: {row['Nulos']:,} nulos ({100 - row['Completude %']:.2f}%)")
    else:
        print(f"Sem valores faltantes (100% completo)")
    
    return report

customers_completeness = analyze_completeness(customers_raw, "CUSTOMERS_RAW")
products_completeness = analyze_completeness(products_raw, "PRODUCTS_RAW")
sales_completeness = analyze_completeness(sales_raw, "SALES_RAW")



ANÁLISE DE COMPLETUDE (Missing Values)

CUSTOMERS_RAW:
Total de linhas: 5,000
Sem valores faltantes (100% completo)

PRODUCTS_RAW:
Total de linhas: 10,000
Sem valores faltantes (100% completo)

SALES_RAW:
Total de linhas: 120,000
Sem valores faltantes (100% completo)


In [4]:
# ===== VALIDAÇÃO DE TIPOS DE DADOS =====
print("\n" + "="*80)
print("VALIDAÇÃO DE TIPOS DE DADOS")
print("="*80)

def analyze_data_types(df, table_name):
    """Analisa os tipos de dados de cada coluna"""
    if df is None:
        return None
    
    print(f"\n{table_name}:")
    
    type_report = df.dtypes.value_counts()
    print(f"Distribuição de tipos:")
    for dtype, count in type_report.items():
        print(f"• {dtype}: {count} campos")
    
    return df.dtypes

customers_types = analyze_data_types(customers_raw, "CUSTOMERS_RAW")
products_types = analyze_data_types(products_raw, "PRODUCTS_RAW")
sales_types = analyze_data_types(sales_raw, "SALES_RAW")



VALIDAÇÃO DE TIPOS DE DADOS

CUSTOMERS_RAW:
Distribuição de tipos:
• object: 8 campos
• float64: 3 campos
• bool: 2 campos

PRODUCTS_RAW:
Distribuição de tipos:
• object: 9 campos
• float64: 3 campos
• int64: 2 campos

SALES_RAW:
Distribuição de tipos:
• object: 9 campos
• int64: 3 campos
• float64: 2 campos


In [5]:
# ===== ANÁLISE DE CHAVES PRIMÁRIAS =====
print("\n" + "="*80)
print("ANÁLISE DE INTEGRIDADE DE CHAVES PRIMÁRIAS")
print("="*80)

def analyze_primary_keys(df, table_name, pk_column):
    """Analisa unicidade de chave primária"""
    if df is None:
        return None
    
    print(f"\n{table_name} (PK: {pk_column}):")
    
    total_rows = len(df)
    unique_ids = df[pk_column].nunique()
    duplicates = total_rows - unique_ids
    
    print(f"Total de registros: {total_rows:,}")
    print(f"IDs únicos: {unique_ids:,}")
    print(f"IDs duplicados: {duplicates}")
    
    if duplicates > 0:
        print(f"AVISO: Encontrados {duplicates} registros duplicados!")
        dup_ids = df[df[pk_column].duplicated(keep=False)][pk_column].unique()
        print(f"IDs duplicados: {list(dup_ids[:5])}{'...' if len(dup_ids) > 5 else ''}")
    else:
        print(f"Sem duplicatas - Integridade OK")
    
    # Verificar NULLs na PK
    null_pks = df[pk_column].isnull().sum()
    if null_pks > 0:
        print(f"{null_pks}registros com PK nula!")
    
    return {'total': total_rows, 'unique': unique_ids, 'duplicates': duplicates}

cust_pk = analyze_primary_keys(customers_raw, "CUSTOMERS_RAW", "customer_id")
prod_pk = analyze_primary_keys(products_raw, "PRODUCTS_RAW", "product_id")
sales_pk = analyze_primary_keys(sales_raw, "SALES_RAW", "sale_id")



ANÁLISE DE INTEGRIDADE DE CHAVES PRIMÁRIAS

CUSTOMERS_RAW (PK: customer_id):
Total de registros: 5,000
IDs únicos: 5,000
IDs duplicados: 0
Sem duplicatas - Integridade OK

PRODUCTS_RAW (PK: product_id):
Total de registros: 10,000
IDs únicos: 10,000
IDs duplicados: 0
Sem duplicatas - Integridade OK

SALES_RAW (PK: sale_id):
Total de registros: 120,000
IDs únicos: 120,000
IDs duplicados: 0
Sem duplicatas - Integridade OK


In [6]:
# ===== INTEGRIDADE REFERENCIAL =====
print("\n" + "="*80)
print("ANÁLISE DE INTEGRIDADE REFERENCIAL (Foreign Keys)")
print("="*80)

if sales_raw is not None and customers_raw is not None and products_raw is not None:
    # FK: sales.customer_id → customers.customer_id
    print(f"\nSALES → CUSTOMERS:")
    invalid_customers = sales_raw[~sales_raw['customer_id'].isin(customers_raw['customer_id'])]
    if len(invalid_customers) > 0:
        print(f"{len(invalid_customers)} registros com customer_id inválido!")
        print(f"Exemplos: {invalid_customers['customer_id'].unique()[:5]}")
    else:
        print(f"Todas as vendas têm customer válido")
    
    # FK: sales.product_id → products.product_id
    print(f"\nSALES → PRODUCTS:")
    invalid_products = sales_raw[~sales_raw['product_id'].isin(products_raw['product_id'])]
    if len(invalid_products) > 0:
        print(f"{len(invalid_products)} registros com product_id inválido!")
        print(f"Exemplos: {invalid_products['product_id'].unique()[:5]}")
    else:
        print(f"Todos as vendas têm product válido")
else:
    print("Não foi possível validar FKs - dados não carregados")



ANÁLISE DE INTEGRIDADE REFERENCIAL (Foreign Keys)

SALES → CUSTOMERS:
Todas as vendas têm customer válido

SALES → PRODUCTS:
Todos as vendas têm product válido


In [7]:
# ===== ANÁLISE DE OUTLIERS =====
print("\n" + "="*80)
print("ANÁLISE DE OUTLIERS - CAMPOS NUMÉRICOS")
print("="*80)

def analyze_outliers(df, table_name, numeric_columns):
    """Detecta outliers usando IQR (Interquartile Range)"""
    if df is None:
        return None
    
    print(f"\n{table_name}:")
    
    outlier_report = {}
    
    for col in numeric_columns:
        if col not in df.columns:
            continue
        
        # Converter para numérico
        data = pd.to_numeric(df[col], errors='coerce')
        
        if data.isnull().all():
            continue
        
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = ((data < lower_bound) | (data > upper_bound)).sum()
        
        if outliers > 0:
            outlier_report[col] = {
                'count': outliers,
                'percentage': (outliers / len(df)) * 100,
                'min': data.min(),
                'max': data.max(),
                'mean': data.mean()
            }
    
    if len(outlier_report) > 0:
        print(f"CAMPOS COM OUTLIERS:")
        for col, info in outlier_report.items():
            print(f"• {col}: {info['count']} outliers ({info['percentage']:.2f}%)")
            print(f"Min: {info['min']:.2f} | Max: {info['max']:.2f} | Média: {info['mean']:.2f}")
    else:
        print(f"Sem outliers detectados")
    
    return outlier_report

# Análise por tabela
cust_outliers = analyze_outliers(
    customers_raw, 
    "CUSTOMERS_RAW",
    ['annual_revenue_estimated', 'maintenance_budget_annual', 'downtime_cost_per_hour']
)

prod_outliers = analyze_outliers(
    products_raw,
    "PRODUCTS_RAW",
    ['load_capacity', 'max_speed', 'temperature_limit', 'unit_cost', 'list_price']
)

sales_outliers = analyze_outliers(
    sales_raw,
    "SALES_RAW",
    ['quantity', 'unit_price', 'total_price', 'discount_percentage']
)



ANÁLISE DE OUTLIERS - CAMPOS NUMÉRICOS

CUSTOMERS_RAW:
Sem outliers detectados

PRODUCTS_RAW:
Sem outliers detectados

SALES_RAW:
CAMPOS COM OUTLIERS:
• total_price: 1419 outliers (1.18%)
Min: 186.88 | Max: 125438.80 | Média: 31644.74


In [8]:
# ===== VALIDAÇÕES DE REGRAS DE NEGÓCIO =====
print("\n" + "="*80)
print("VALIDAÇÕES DE REGRAS DE NEGÓCIO")
print("="*80)

# ===== CUSTOMERS =====
print("\nCUSTOMERS_RAW:")

if customers_raw is not None:
    # maintenance_budget_annual deve ser positivo
    negative_budget = (customers_raw['maintenance_budget_annual'] < 0).sum()
    if negative_budget > 0:
        print(f"{negative_budget} registros com orçamento negativo!")
    else:
        print(f"Todos orçamentos são positivos")
    
    # Regra 2: downtime_cost_per_hour deve ser positivo
    negative_downtime = (customers_raw['downtime_cost_per_hour'] < 0).sum()
    if negative_downtime > 0:
        print(f"{negative_downtime} registros com custo de downtime negativo!")
    else:
        print(f"Todos custos de downtime são positivos")
    
    # Regra 3: relationship_start_date não pode ser no futuro
    customers_raw['relationship_start_date'] = pd.to_datetime(customers_raw['relationship_start_date'])
    future_dates = (customers_raw['relationship_start_date'] > datetime.now()).sum()
    if future_dates > 0:
        print(f"{future_dates} registros com data de início no futuro!")
    else:
        print(f"Datas de relacionamento são válidas")

# ===== PRODUCTS =====
print("\nPRODUCTS_RAW:")

if products_raw is not None:
    # Regra 1: list_price deve ser maior que unit_cost
    negative_margin = (products_raw['list_price'] <= products_raw['unit_cost']).sum()
    if negative_margin > 0:
        print(f"{negative_margin} produtos com margem negativa!")
        problematic_products = products_raw[products_raw['list_price'] <= products_raw['unit_cost']]
        print(f"Exemplos:")
        for idx, row in problematic_products.head(3).iterrows():
            print(f"• {row['product_id']}: Custo={row['unit_cost']:.2f}, Preço={row['list_price']:.2f}")
    else:
        print(f"Todas margens são positivas")
    
    # Regra 2: max_speed deve ser positivo
    zero_speed = (products_raw['max_speed'] <= 0).sum()
    if zero_speed > 0:
        print(f"{zero_speed} produtos com velocidade zero/negativa!")
    else:
        print(f"Todas velocidades são positivas")
    
    # Regra 3: load_capacity deve ser positivo
    zero_load = (products_raw['load_capacity'] <= 0).sum()
    if zero_load > 0:
        print(f"{zero_load} produtos com capacidade de carga zero/negativa!")
    else:
        print(f"Todas capacidades de carga são positivas")

# ===== SALES =====
print("\nSALES_RAW:")

if sales_raw is not None:
    # Regra 1: total_price deve ser aproximadamente quantity * unit_price
    sales_raw['quantity'] = pd.to_numeric(sales_raw['quantity'], errors='coerce')
    sales_raw['unit_price'] = pd.to_numeric(sales_raw['unit_price'], errors='coerce')
    sales_raw['total_price'] = pd.to_numeric(sales_raw['total_price'], errors='coerce')
    
    expected_total = sales_raw['quantity'] * sales_raw['unit_price']
    price_mismatch = (abs(sales_raw['total_price'] - expected_total) > 0.01).sum()
    if price_mismatch > 0:
        print(f"{price_mismatch} registros com total_price inconsistente!")
    else:
        print(f"Total_price validado corretamente")
    
    # Regra 2: discount_percentage deve estar entre 0 e 100
    invalid_discount = ((sales_raw['discount_percentage'] < 0) | (sales_raw['discount_percentage'] > 100)).sum()
    if invalid_discount > 0:
        print(f"{invalid_discount} registros com desconto fora do range [0-100]!")
    else:
        print(f"Todos descontos estão no range válido")
    
    # Regra 3: sale_date deve ser válida
    sales_raw['sale_date'] = pd.to_datetime(sales_raw['sale_date'], errors='coerce')
    invalid_dates = sales_raw['sale_date'].isnull().sum()
    if invalid_dates > 0:
        print(f"{invalid_dates} registros com data inválida!")
    else:
        print(f"Todas datas são válidas")



VALIDAÇÕES DE REGRAS DE NEGÓCIO

CUSTOMERS_RAW:
Todos orçamentos são positivos
Todos custos de downtime são positivos
Datas de relacionamento são válidas

PRODUCTS_RAW:
359 produtos com margem negativa!
Exemplos:
• P00001: Custo=315.72, Preço=288.99
• P00023: Custo=386.59, Preço=360.06
• P00038: Custo=368.95, Preço=204.74
Todas velocidades são positivas
Todas capacidades de carga são positivas

SALES_RAW:
Total_price validado corretamente
Todos descontos estão no range válido
Todas datas são válidas


In [9]:
# ===== ESTATÍSTICAS DESCRITIVAS =====
print("\n" + "="*80)
print("ANÁLISE DE DISTRIBUIÇÃO - ESTATÍSTICAS DESCRITIVAS")
print("="*80)

def print_stats(df, table_name, columns):
    """Imprime estatísticas descritivas"""
    if df is None:
        return
    
    print(f"\n{table_name}:")
    
    for col in columns:
        if col not in df.columns:
            continue
        
        data = pd.to_numeric(df[col], errors='coerce')
        if data.isnull().all():
            continue
        
        print(f"\n{col}:")
        print(f"Count: {data.count():,}")
        print(f"Mean: {data.mean():.2f}")
        print(f"Std: {data.std():.2f}")
        print(f"Min: {data.min():.2f}")
        print(f"P25: {data.quantile(0.25):.2f}")
        print(f"Median: {data.quantile(0.5):.2f}")
        print(f"P75: {data.quantile(0.75):.2f}")
        print(f"Max: {data.max():.2f}")

print_stats(
    customers_raw,
    "CUSTOMERS_RAW",
    ['annual_revenue_estimated', 'maintenance_budget_annual', 'downtime_cost_per_hour']
)

print_stats(
    sales_raw,
    "SALES_RAW",
    ['quantity', 'unit_price', 'total_price', 'discount_percentage']
)



ANÁLISE DE DISTRIBUIÇÃO - ESTATÍSTICAS DESCRITIVAS

CUSTOMERS_RAW:

annual_revenue_estimated:
Count: 5,000
Mean: 2480102761.04
Std: 1444790522.56
Min: 6388020.53
P25: 1222530616.29
Median: 2449600585.93
P75: 3759988026.06
Max: 4997395539.30

maintenance_budget_annual:
Count: 5,000
Mean: 2528331.92
Std: 1421248.53
Min: 50188.42
P25: 1293402.82
Median: 2521822.10
P75: 3737594.74
Max: 4999969.91

downtime_cost_per_hour:
Count: 5,000
Mean: 25721.16
Std: 14086.77
Min: 1013.02
P25: 13443.51
Median: 25671.33
P75: 38187.65
Max: 49999.25

SALES_RAW:

quantity:
Count: 120,000
Mean: 20.51
Std: 11.52
Min: 1.00
P25: 11.00
Median: 21.00
P75: 30.00
Max: 40.00

unit_price:
Count: 120,000
Mean: 1543.27
Std: 791.53
Min: 183.09
P25: 859.05
Median: 1533.95
P75: 2217.11
Max: 3147.48

total_price:
Count: 120,000
Mean: 31644.74
Std: 25741.18
Min: 186.88
P25: 10621.11
Median: 24707.56
P75: 47089.82
Max: 125438.80

discount_percentage:
Count: 120,000
Mean: 7.50
Std: 5.59
Min: 0.00
P25: 0.00
Median: 10.00
P75:

In [10]:
# ===== RESUMO FINAL =====
print("\n" + "="*80)
print("RELATÓRIO - DATA QUALITY SUMMARY")
print("="*80)

summary = f"""
DATA: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}
FASE: Data Quality Assessment - Raw Layer com Criptografia (Dadosfera)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
VOLUME DE DADOS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
"""

if customers_raw is not None:
    summary += f"customers_raw: {len(customers_raw):>8,} registros | {len(customers_raw.columns):>2} campos\n"
if products_raw is not None:
    summary += f"products_raw: {len(products_raw):>8,} registros | {len(products_raw.columns):>2} campos\n"
if sales_raw is not None:
    summary += f"sales_raw: {len(sales_raw):>8,} registros | {len(sales_raw.columns):>2} campos\n"

total_records = (len(customers_raw) if customers_raw is not None else 0) + \
                (len(products_raw) if products_raw is not None else 0) + \
                (len(sales_raw) if sales_raw is not None else 0)
summary += f"\nTOTAL: {total_records:,} registros\n"

summary += f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ÍNDICES DE QUALIDADE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

COMPLETUDE (Missing Values):
  • Sem dados faltantes: (100%)

INTEGRIDADE DE CHAVES PRIMÁRIAS:
  • Sem duplicatas: (100%)
  • customers.customer_id: Único
  • products.product_id: Único
  • sales.sale_id: Único

INTEGRIDADE REFERENCIAL:
  • sales.customer_id → customers: Válido
  • sales.product_id → products: Válido

OUTLIERS:
  • Detectados via IQR: Consultar análise acima!

REGRAS DE NEGÓCIO:
  • Margens de lucro: Verificar análise!
  • Descontos válidos: (100%)
  • Cálculos de total_price: (100%)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
CAMPOS CRIPTOGRAFADOS (Dadosfera)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

CUSTOMERS:
  • company_name
  • annual_revenue_estimated
  • maintenance_budget_annual
  • downtime_cost_per_hour

PRODUCTS:
  • unit_cost
  • list_price

SALES:
  • customer_id
  • unit_price
  • total_price
  • discount_percentage

"""

print(summary)
print("\n" + "="*80)
print(f"Relatório gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)



RELATÓRIO - DATA QUALITY SUMMARY

DATA: 10/01/2026 18:53:23
FASE: Data Quality Assessment - Raw Layer com Criptografia (Dadosfera)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
VOLUME DE DADOS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
customers_raw:    5,000 registros | 13 campos
products_raw:   10,000 registros | 14 campos
sales_raw:  120,000 registros | 14 campos

TOTAL: 135,000 registros

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
ÍNDICES DE QUALIDADE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

COMPLETUDE (Missing Values):
  • Sem dados faltantes: (100%)

INTEGRIDADE DE CHAVES PRIMÁRIAS:
  • Sem duplicatas: (100%)
  • customers.customer_id: Único
  • products.product_id: Único
  • sales.sale_id: Único

INTEGRIDADE REFERENCIAL:
  • sales.customer_id → customers: Válido
  • sales.product_id → products: Válido

OUTLIERS:
  • Detectados via IQR: